In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
import matplotlib.pyplot as plt

In [2]:
# 랜덤에 의해 똑같은 결과를 재현하도록 시드 설정
# 하이퍼파라미터를 튜닝하기 위한 용도(흔들리면 무엇때문에 좋아졌는지 알기 어려움)
tf.set_random_seed(777)

In [3]:
def MinMaxScaler(data):
    # 데이터 모든숫자들을 최소 값만큼 뺀다.
    numerator = data - np.min(data, 0)
    # 최대값과 최소 값의 차이(A)를 구한다
    denominator = np.max(data, 0) - np.min(data, 0)
    # 너무 큰 값이 나오지 않도록 나눈다
    return numerator / (denominator + 1e-7)

In [18]:
# 하이퍼파라미터
seq_length = 7       # 1개 시퀀스의 길이(시계열데이터 입력 개수)
data_dim = 5         # Variable 개수
hidden_dim = 10      # 각 셀의 출력 크기
output_dim = 1       # 결과 분류 총 수
learning_rate = 0.01 # 학습률
epoch_num = 500      # 에폭 횟수(학습용전체데이터를 몇 회 반복해서 학습할 것인가 입력)

In [12]:
# 데이터를 로딩한다.
# 시작가, 고가, 저가, 거래량, 종가
xy = np.loadtxt('stock_daily_price.csv', delimiter=',')
import pandas as pd
xy

array([[8.28659973e+02, 8.33450012e+02, 8.28349976e+02, 1.24770000e+06,
        8.31659973e+02],
       [8.23020020e+02, 8.28070007e+02, 8.21655029e+02, 1.59780000e+06,
        8.28070007e+02],
       [8.19929993e+02, 8.24400024e+02, 8.18979980e+02, 1.28170000e+06,
        8.24159973e+02],
       ...,
       [5.66892592e+02, 5.67002574e+02, 5.56932537e+02, 1.08000000e+04,
        5.56972503e+02],
       [5.61202549e+02, 5.66432590e+02, 5.58672539e+02, 4.12000000e+04,
        5.59992565e+02],
       [5.68002570e+02, 5.68002570e+02, 5.52922516e+02, 1.31000000e+04,
        5.58462551e+02]])

In [19]:
# 데이터 전처리
xy = xy[::-1] # 제일앞이 뒤로, 제일뒤가 앞으로 순서를 뒤집는다.
print("xy[0][0]: ", xy[0][0])
xy = MinMaxScaler(xy)
print("xy[0][0]: ", xy[0][0])
x = xy
y = xy[:, [-1]] # 마지막 열이 정답(주식 종가)이다.
print("x[0]: ", x[0])
print("y[0]: ",y[0])
y.ndim
x

xy[0][0]:  0.9733357089925367
xy[0][0]:  0.9733357089925367
x[0]:  [0.97333571 0.97543142 0.9999999  0.11112305 0.98831292]
y[0]:  [0.98831292]


array([[9.73335709e-01, 9.75431425e-01, 9.99999900e-01, 1.11123051e-01,
        9.88312922e-01],
       [9.56900258e-01, 9.59881011e-01, 9.80354399e-01, 1.42502451e-01,
        9.77850141e-01],
       [9.47895576e-01, 9.49273259e-01, 9.72504796e-01, 1.14170465e-01,
        9.66454538e-01],
       ...,
       [2.10516433e-01, 2.05289392e-01, 2.03558728e-01, 2.59926478e-04,
        1.87749712e-01],
       [1.93935014e-01, 2.03641906e-01, 2.08664550e-01, 2.98467300e-03,
        1.96551535e-01],
       [2.13751032e-01, 2.08179790e-01, 1.91791813e-01, 4.66075063e-04,
        1.92092384e-01]])

In [20]:
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i : i+seq_length]
    _y = y[i + seq_length] # 다음 나타날 주가(정답)
    if i is 0:
        print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)
dataX
dataY

[[0.97333571 0.97543142 0.9999999  0.11112305 0.98831292]
 [0.95690026 0.95988101 0.9803544  0.14250245 0.97785014]
 [0.94789558 0.94927326 0.9725048  0.11417046 0.96645454]
 [0.94623451 0.94522662 0.97100824 0.11616921 0.9513577 ]
 [0.94518547 0.94522662 0.96376041 0.0937259  0.95564204]
 [0.93644314 0.93932725 0.96226385 0.10667741 0.95211549]
 [0.9239125  0.92282595 0.94550867 0.10049295 0.93588197]] -> [0.92390362]


[array([0.92390362]),
 array([0.92046459]),
 array([0.91635512]),
 array([0.89994692]),
 array([0.90038398]),
 array([0.89175732]),
 array([0.88349479]),
 array([0.88668603]),
 array([0.90280302]),
 array([0.96397733]),
 array([0.98974115]),
 array([0.9999999]),
 array([0.96560941]),
 array([0.95231952]),
 array([0.91067208]),
 array([0.90238037]),
 array([0.91373222]),
 array([0.90947705]),
 array([0.91900737]),
 array([0.91457734]),
 array([0.91909471]),
 array([0.91000163]),
 array([0.91542265]),
 array([0.91396542]),
 array([0.87861312]),
 array([0.85786224]),
 array([0.85564723]),
 array([0.81391226]),
 array([0.84588371]),
 array([0.8524704]),
 array([0.87141433]),
 array([0.86663459]),
 array([0.8705692]),
 array([0.88018686]),
 array([0.88560769]),
 array([0.8791377]),
 array([0.86922849]),
 array([0.88977534]),
 array([0.88750216]),
 array([0.88467505]),
 array([0.86476947]),
 array([0.86482764]),
 array([0.82731867]),
 array([0.81207615]),
 array([0.77686953]),
 array([0.7868

In [8]:
# 학습용/테스트용 데이터 생성
# 70%를 학습용 데이터로 사용
train_size = int(len(dataY) * 0.7)
# 나머지(30%)를 테스트용 데이터로 사용
test_size = len(dataY) - train_size

# 데이터를 잘라 학습용 데이터 생성
trainX = np.array(dataX[0:train_size])
trainY = np.array(dataY[0:train_size])

# 데이터를 잘라 테스트용 데이터 생성
testX = np.array(dataX[train_size:len(dataX)])
testY = np.array(dataY[train_size:len(dataY)])

In [9]:
# 텐서플로우 플레이스홀더 생성
# 학습용/테스트용으로 X, Y를 생성한다
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
print("X: ", X)
Y = tf.placeholder(tf.float32, [None, 1])
print("Y: ", Y)

# 검증용 측정지표를 산출하기 위한 targets, predictions를 생성한다
targets = tf.placeholder(tf.float32, [None, 1])
print("targets: ", targets)
predictions = tf.placeholder(tf.float32, [None, 1])
print("predictions: ", predictions)

X:  Tensor("Placeholder:0", shape=(?, 7, 5), dtype=float32)
Y:  Tensor("Placeholder_1:0", shape=(?, 1), dtype=float32)
targets:  Tensor("Placeholder_2:0", shape=(?, 1), dtype=float32)
predictions:  Tensor("Placeholder_3:0", shape=(?, 1), dtype=float32)


In [10]:
# 모델(LSTM 네트워크) 생성
def lstm_cell():
    # LSTM셀을 생성한다.
    # num_units: 각 Cell 출력 크기
    # forget_bias: The bias added to forget gates.
    # state_is_tuple: True ==> accepted and returned states are 2-tuples of the c_state and m_state.
    # state_is_tuple: False ==> they are concatenated along the column axis.
    # cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.sigmoid)
    # cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, forget_bias=0.8, state_is_tuple=True, activation=tf.tanh)
    return cell

# 몇개의 층으로 쌓인 Stacked RNNs 생성, 여기서는 1개층만
multi_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(1)], state_is_tuple=True)

# RNN Cell(여기서는 LSTM셀임)들을 연결
hypothesis, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
print("hypothesis: ", hypothesis)

Y_pred = tf.contrib.layers.fully_connected(hypothesis[:, -1], output_dim, activation_fn=None)

hypothesis:  Tensor("rnn/transpose:0", shape=(?, 7, 10), dtype=float32)


In [11]:
loss = tf.reduce_sum(tf.square(Y_pred - Y))
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

# RMSE(Root Mean Square Error)
# rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions))) # 아래 코드와 같다
rmse = tf.sqrt(tf.reduce_mean(tf.squared_difference(targets, predictions)))

In [22]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # 학습한다
    for epoch in range(epoch_num):
        _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
        print("[step: {} loss: {}".format(epoch, step_loss))
    
    # 테스트한다
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    
    # 테스트용 데이터 기준으로 측정지표 rmse를 산출한다
    rmse_val = sess.run(rmse, feed_dict={targets: testY, predictions: test_predict})
    print("rmse: ", rmse_val)
    
    plt.plot(testY, 'r')
    plt.plot(test_predict, 'b')
    plt.xlabel("Time Period")
    plt.ylabel("Stock Price")
    plt.show()

NameError: name 'train' is not defined

In [21]:
tf.reset_default_graph()